In [1]:
import glob
import pandas as pd
import numpy as np
from numpy.linalg import norm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dropout
import tensorflow as tf
from tqdm import tqdm
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer

%matplotlib inline

2024-03-01 06:24:48.566488: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 06:24:48.641715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 06:24:48.641772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 06:24:48.645411: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 06:24:48.659530: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 06:24:48.660476: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
dict_2classes = {}
dict_2classes['DDoS-RSTFINFlood'] = 'Attack'
dict_2classes['DDoS-PSHACK_Flood'] = 'Attack'
dict_2classes['DDoS-SYN_Flood'] = 'Attack'
dict_2classes['DDoS-UDP_Flood'] = 'Attack'
dict_2classes['DDoS-TCP_Flood'] = 'Attack'
dict_2classes['DDoS-ICMP_Flood'] = 'Attack'
dict_2classes['DDoS-SynonymousIP_Flood'] = 'Attack'
dict_2classes['DDoS-ACK_Fragmentation'] = 'Attack'
dict_2classes['DDoS-UDP_Fragmentation'] = 'Attack'
dict_2classes['DDoS-ICMP_Fragmentation'] = 'Attack'
dict_2classes['DDoS-SlowLoris'] = 'Attack'
dict_2classes['DDoS-HTTP_Flood'] = 'Attack'

dict_2classes['DoS-UDP_Flood'] = 'Attack'
dict_2classes['DoS-SYN_Flood'] = 'Attack'
dict_2classes['DoS-TCP_Flood'] = 'Attack'
dict_2classes['DoS-HTTP_Flood'] = 'Attack'


dict_2classes['Mirai-greeth_flood'] = 'Attack'
dict_2classes['Mirai-greip_flood'] = 'Attack'
dict_2classes['Mirai-udpplain'] = 'Attack'

dict_2classes['Recon-PingSweep'] = 'Attack'
dict_2classes['Recon-OSScan'] = 'Attack'
dict_2classes['Recon-PortScan'] = 'Attack'
dict_2classes['VulnerabilityScan'] = 'Attack'
dict_2classes['Recon-HostDiscovery'] = 'Attack'

dict_2classes['DNS_Spoofing'] = 'Attack'
dict_2classes['MITM-ArpSpoofing'] = 'Attack'

dict_2classes['BenignTraffic'] = 'Benign'

dict_2classes['BrowserHijacking'] = 'Attack'
dict_2classes['Backdoor_Malware'] = 'Attack'
dict_2classes['XSS'] = 'Attack'
dict_2classes['Uploading_Attack'] = 'Attack'
dict_2classes['SqlInjection'] = 'Attack'
dict_2classes['CommandInjection'] = 'Attack'

dict_2classes['DictionaryBruteForce'] = 'Attack'

In [3]:

file_paths = glob.glob('/home/haohao/Downloads/archive/wataiData/csv/CICIoT2023/*.csv')
file_paths.sort()
print('file_paths len: ', len(file_paths))

df = pd.DataFrame()
#[:round(len(file_paths)*0.01)]
for file_path in tqdm(file_paths[:round(len(file_paths)*0.01)], desc='Processing files', unit='file'):
    dff = pd.read_csv(file_path)
    
    df = pd.concat([df, dff], ignore_index=True)

file_paths len:  169


Processing files:   0%|          | 0/2 [00:00<?, ?file/s]

Processing files: 100%|██████████| 2/2 [00:02<00:00,  1.44s/file]


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457492 entries, 0 to 457491
Data columns (total 47 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   flow_duration    457492 non-null  float64
 1   Header_Length    457492 non-null  float64
 2   Protocol Type    457492 non-null  float64
 3   Duration         457492 non-null  float64
 4   Rate             457492 non-null  float64
 5   Srate            457492 non-null  float64
 6   Drate            457492 non-null  float64
 7   fin_flag_number  457492 non-null  float64
 8   syn_flag_number  457492 non-null  float64
 9   rst_flag_number  457492 non-null  float64
 10  psh_flag_number  457492 non-null  float64
 11  ack_flag_number  457492 non-null  float64
 12  ece_flag_number  457492 non-null  float64
 13  cwr_flag_number  457492 non-null  float64
 14  ack_count        457492 non-null  float64
 15  syn_count        457492 non-null  float64
 16  fin_count        457492 non-null  floa

In [5]:
df['label'] = df['label'].map(dict_2classes)
df = df.sample(frac=1, random_state=42)

In [6]:
df.head(10)

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
413784,0.130570,30076.00,17.00,64.00,4580.297292,4580.297292,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.310187e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,Attack
433656,0.000000,0.00,1.00,64.00,37.205150,37.205150,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.314968e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,Attack
148341,0.000000,54.00,6.00,64.00,47.117104,47.117104,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.309008e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
314309,0.015934,62.28,6.00,65.72,31.289158,31.289158,0.0,1.0,0.0,1.0,...,3.514623,54.59,8.334500e+07,9.5,10.532031,4.974893,69.757079,0.19,141.55,Attack
66880,0.000000,53.46,5.94,63.36,0.602080,0.602080,0.0,0.0,0.0,0.0,...,0.249154,54.06,8.333107e+07,9.5,10.399888,0.352811,0.426318,0.15,141.55,Attack
160088,0.000000,54.00,6.00,64.00,3.005428,3.005428,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.309440e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
94890,0.064326,167.40,16.84,64.00,2.415185,2.415185,0.0,0.0,0.0,0.0,...,3.037121,161.24,8.300769e+07,9.5,17.965543,4.303642,84.921741,0.11,141.55,Attack
284143,0.012008,57.16,6.00,64.00,9.014776,9.014776,0.0,0.0,0.0,0.0,...,0.330928,54.20,8.307202e+07,9.5,10.400347,0.469517,1.578792,0.07,141.55,Attack
140498,0.000000,54.00,6.00,64.00,43.054271,43.054271,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.295084e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
61862,0.000000,0.00,1.00,64.00,3.764777,3.764777,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.312811e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,Attack


In [7]:
y = pd.get_dummies(df['label']).iloc[:, ::-1].to_numpy()

y_labels = np.argmax(y, axis=1)

print(y[0:10])
print(df['label'].head(10))
y_labels[:10]

[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
413784    Attack
433656    Attack
148341    Attack
314309    Attack
66880     Attack
160088    Attack
94890     Attack
284143    Attack
140498    Attack
61862     Attack
Name: label, dtype: object


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [8]:

# Đếm số lượng mỗi nhãn
label_counts = np.bincount(y_labels)

# In ra số lượng mỗi nhãn
for i, count in enumerate(label_counts):
    print(f"Label {i}: {count} occurrences")

Label 0: 10800 occurrences
Label 1: 446692 occurrences


In [9]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(method='ffill', inplace=True)
scaler = MinMaxScaler()
X = df.iloc[:, :-1]
X = scaler.fit_transform(X)

In [10]:
X[0:10]

array([[1.90806842e-06, 3.06411609e-03, 3.61702128e-01, 2.50980392e-01,
        6.24015984e-04, 6.24015984e-04, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 7.81040287e-03, 2.37079184e-03, 2.64137986e-04,
        1.02313499e-03, 0.00000000e+00, 6.12745098e-04, 4.95717949e-01,
        6.29629630e-01, 7.46665227e-03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 5.76970443e-01],
       [0.00000000e+00, 0.00000000e+00, 2.12765957e-02, 2.50980392e-01,
        5.06879945e-06, 5.06879945e-06, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00,

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y_labels, test_size=0.2, random_state=75)

In [12]:

# Đếm số lượng mỗi nhãn
label_counts = np.bincount(y_test)
#label_counts = np.bincount(y_train)

# In ra số lượng mỗi nhãn
for i, count in enumerate(label_counts):
    print(f"Label {i}: {count} occurrences")

Label 0: 2153 occurrences
Label 1: 89346 occurrences


In [13]:
from sklearn.metrics import classification_report, confusion_matrix

def eval(model):
    # Dự đoán nhãn trên tập kiểm thử
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    y_pred = tf.squeeze(y_pred, axis=1)
    print(y_pred[:10])
    print(y_test[:10])
    cm = confusion_matrix(y_test, y_pred)
    # Tính toán độ chính xác tổng thể
    total_accuracy = (cm[0, 0] + cm[1, 1]) / np.sum(cm)


    # Tính toán F1-score, Recall và Precision
    report = classification_report(y_test, y_pred, target_names=None)

    # Thêm vào báo cáo
    report += '\nTotal accuracy: {:0.2f} \n'.format(total_accuracy)

    print(report)


In [14]:
def build_model():
  n_features = X_train.shape[1]
  model = Sequential()
  model.add(Dense(512, input_dim=n_features, activation='relu'))  # Input layer và hidden layer đầu tiên
  model.add(Dropout(0.3))  # Thêm dropout để giảm overfitting
  model.add(Dense(256, activation='relu'))  # Thêm hidden layer thứ hai
  model.add(Dropout(0.3))  # Thêm dropout
  model.add(Dense(128, activation='relu'))  # Thêm hidden layer thứ hai
  model.add(Dropout(0.3))  # Thêm dropout
  model.add(Dense(64, activation='relu'))  # Thêm hidden layer thứ hai
  model.add(Dropout(0.3))  # Thêm dropout
  model.add(Dense(1, activation='sigmoid'))  # Output layer

  # Compile mô hình
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [15]:
'''
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = build_model()

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

eval(model)
'''

'\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense, Dropout\n\nmodel = build_model()\n\n# Huấn luyện mô hình\nmodel.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))\n\neval(model)\n'

In [16]:
'''
eval(model)
'''

'\neval(model)\n'

In [17]:
# Tính tổng số mẫu

n_samples = X_train.shape[0]
print(n_samples)

# Tính chỉ số cho việc chia tách
# Phần 1: 0 -> 5/10 * n_samples
# Phần 2: 5/10 * n_samples -> (5+3)/10 * n_samples
# Phần 3: (5+3)/10 * n_samples -> n_samples
indices = [int(5/10 * n_samples), int((5+3)/10 * n_samples)]

# Chia dữ liệu
client_data = np.split(X_train, indices)
client_labels = np.split(y_train, indices)

# client_data[0], client_labels[0] sẽ chứa 50% dữ liệu đầu tiên
# client_data[1], client_labels[1] sẽ chứa 30% dữ liệu tiếp theo
# client_data[2], client_labels[2] sẽ chứa 20% dữ liệu cuối cùng


365993


In [18]:
'''
import numpy as np
n_samples = X_train.shape[0]
# Giả sử X_train, y_train là dữ liệu huấn luyện của bạn
# n_samples = X_train.shape[0]

# Định nghĩa tỷ lệ cho mỗi node
ratios = [0.3, 0.3, 0.4]  # Tổng cộng phải bằng 1

# Tính chỉ số cho việc chia tách dựa trên tỷ lệ
indices = [int(sum(ratios[:i+1]) * n_samples) for i in range(len(ratios)-1)]

# Chia dữ liệu
client_data = np.split(X_train, indices)
client_labels = np.split(y_train, indices)

# In kết quả để kiểm tra
for i, (data, labels) in enumerate(zip(client_data, client_labels)):
    print(f"Node {i+1}:")
    print(f" - Dữ liệu: {data.shape}")
    print(f" - Nhãn: {labels.shape}")

'''

'\nimport numpy as np\nn_samples = X_train.shape[0]\n# Giả sử X_train, y_train là dữ liệu huấn luyện của bạn\n# n_samples = X_train.shape[0]\n\n# Định nghĩa tỷ lệ cho mỗi node\nratios = [0.3, 0.3, 0.4]  # Tổng cộng phải bằng 1\n\n# Tính chỉ số cho việc chia tách dựa trên tỷ lệ\nindices = [int(sum(ratios[:i+1]) * n_samples) for i in range(len(ratios)-1)]\n\n# Chia dữ liệu\nclient_data = np.split(X_train, indices)\nclient_labels = np.split(y_train, indices)\n\n# In kết quả để kiểm tra\nfor i, (data, labels) in enumerate(zip(client_data, client_labels)):\n    print(f"Node {i+1}:")\n    print(f" - Dữ liệu: {data.shape}")\n    print(f" - Nhãn: {labels.shape}")\n\n'

In [19]:
def fed_avg(models, data_distribution):
    """
    models: danh sách các mô hình từ các node
    data_distribution: danh sách chứa số lượng dữ liệu mỗi node có
    """
    # Tính tổng số mẫu dữ liệu
    total_data = sum(data_distribution)

    # Khởi tạo trọng số trung bình
    avg_weights = [np.zeros_like(w) for w in models[0].get_weights()]

    # Cộng trọng số từ tất cả mô hình, đã được nhân với tỷ lệ dữ liệu tương ứng
    for model, data_count in zip(models, data_distribution):
        for idx, layer_weights in enumerate(model.get_weights()):
            avg_weights[idx] += (layer_weights * data_count) / total_data

    return avg_weights


In [20]:
global_model = build_model()

In [21]:
# Các thông số DP
l2_norm_clip = 1.0
noise_multiplier = 0.4


# FL
num_rounds = 50
num_epochs = 3
batch_size = 128

In [22]:
"""from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod
import numpy as np

# Xác định mô hình và định nghĩa bộ phân loại ART
model = build_model()  # Sử dụng hàm build_model bạn đã cung cấp
classifier = TensorFlowV2Classifier(model=model, nb_classes=2, input_shape=(X_train.shape[1],), loss_object=tf.keras.losses.BinaryCrossentropy(), clip_values=(0, 1))

# Tạo một instance của FGSM
attack = FastGradientMethod(estimator=classifier, eps=0.1)

# Chọn một phần của tập dữ liệu để tạo dữ liệu đầu độc
# Sử dụng một phần nhỏ của dữ liệu để tránh tác động quá lớn đến quá trình huấn luyện
indices = np.random.choice(len(X_train), size=100000, replace=False)
X_train_subset = X_train[indices]
y_train_subset = y_train[indices]

# Tạo dữ liệu đầu độc
poisoned_X = attack.generate(x=X_train_subset)
poisoned_y = y_train_subset  # Giữ nguyên nhãn của các mẫu đã chọn

# Bây giờ `poisoned_X` và `poisoned_y` là dữ liệu và nhãn đầu độc
"""

'from art.estimators.classification import TensorFlowV2Classifier\nfrom art.attacks.evasion import FastGradientMethod\nimport numpy as np\n\n# Xác định mô hình và định nghĩa bộ phân loại ART\nmodel = build_model()  # Sử dụng hàm build_model bạn đã cung cấp\nclassifier = TensorFlowV2Classifier(model=model, nb_classes=2, input_shape=(X_train.shape[1],), loss_object=tf.keras.losses.BinaryCrossentropy(), clip_values=(0, 1))\n\n# Tạo một instance của FGSM\nattack = FastGradientMethod(estimator=classifier, eps=0.1)\n\n# Chọn một phần của tập dữ liệu để tạo dữ liệu đầu độc\n# Sử dụng một phần nhỏ của dữ liệu để tránh tác động quá lớn đến quá trình huấn luyện\nindices = np.random.choice(len(X_train), size=100000, replace=False)\nX_train_subset = X_train[indices]\ny_train_subset = y_train[indices]\n\n# Tạo dữ liệu đầu độc\npoisoned_X = attack.generate(x=X_train_subset)\npoisoned_y = y_train_subset  # Giữ nguyên nhãn của các mẫu đã chọn\n\n# Bây giờ `poisoned_X` và `poisoned_y` là dữ liệu và nhã

In [23]:
import numpy as np

# Giả sử `X_train` và `y_train` là dữ liệu và nhãn huấn luyện ban đầu

# Chọn một phần nhỏ của tập dữ liệu để thay đổi nhãn
num_poisoned_samples = int(0.3 * len(X_train))  # Ví dụ: 10% dữ liệu
poisoned_indices = np.random.choice(len(X_train), num_poisoned_samples, replace=False)

# Sao chép dữ liệu và nhãn ban đầu để tránh thay đổi trực tiếp
poisoned_X = np.copy(X_train[poisoned_indices])
poisoned_y = np.copy(y_train[poisoned_indices])

# Thay đổi nhãn của dữ liệu đã chọn
# Ví dụ: đối với bài toán phân loại nhị phân, chúng ta có thể đảo ngược nhãn
poisoned_y = 1 - poisoned_y

# Bây giờ `poisoned_X` và `poisoned_y` là dữ liệu và nhãn đã bị đầu độc
print(len(poisoned_X))
print(len(poisoned_y))

109797
109797


In [24]:
for round in range(3):
    local_models = []
    cnt = 1
    for client_X, client_y in zip(client_data, client_labels):
        print('Round ', round, ' - Client ', cnt)
        cnt += 1
        # Sao chép trọng số từ global model
        local_model = build_model()
        local_model.set_weights(global_model.get_weights())
        
        # Biên dịch mô hình với bộ tối ưu hóa DP
        local_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


        # Huấn luyện mô hình cục bộ
        local_model.fit(client_X, client_y, epochs=num_epochs, batch_size=batch_size, verbose=0)
        local_models.append(local_model)

    # Cập nhật global model dựa trên trung bình trọng số từ các mô hình cục bộ
    global_model.set_weights(fed_avg(local_models, [len(cli_dt) for cli_dt in client_data]))

    print('Round ', round, ' - Evaluation:')
    eval(global_model)

Round  0  - Client  1
Round  0  - Client  2
Round  0  - Client  3
Round  0  - Evaluation:
2860/2860 [==============================] - 6s 2ms/step
tf.Tensor([1 1 1 1 1 1 1 1 1 1], shape=(10,), dtype=int64)
[1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.65      0.99      0.79      2153
           1       1.00      0.99      0.99     89346

    accuracy                           0.99     91499
   macro avg       0.83      0.99      0.89     91499
weighted avg       0.99      0.99      0.99     91499

Total accuracy: 0.99 

Round  1  - Client  1
Round  1  - Client  2
Round  1  - Client  3
Round  1  - Evaluation:
2860/2860 [==============================] - 6s 2ms/step
tf.Tensor([1 1 1 1 1 1 1 1 1 1], shape=(10,), dtype=int64)
[1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.71      0.95      0.81      2153
           1       1.00      0.99      0.99     89346

    accuracy                  

In [25]:
# Bước 2: Thêm dữ liệu đầu độc vào node độc hại
# Giả sử 'poisoned_X' và 'poisoned_y' là dữ liệu và nhãn đầu độc
client_data.append(poisoned_X)  # Thêm dữ liệu đầu độc
client_labels.append(poisoned_y)  # Thêm nhãn đầu độc

def cosine_similarity(weights_a, weights_b):
    dot_product = np.sum([np.dot(a.flatten(), b.flatten()) for a, b in zip(weights_a, weights_b)])
    norm_a = np.sqrt(np.sum([np.dot(a.flatten(), a.flatten()) for a in weights_a]))
    norm_b = np.sqrt(np.sum([np.dot(b.flatten(), b.flatten()) for b in weights_b]))
    return dot_product / (norm_a * norm_b)

# Giả định global_weights là trọng số của mô hình toàn cầu từ vòng lặp trước
global_weights_previous_round = global_model.get_weights()

for round in range(num_rounds):
    local_models = []
    similarities = []
    for client_X, client_y in zip(client_data, client_labels):
        # Huấn luyện mô hình cục bộ như bình thường
        local_model = build_model()
        local_model.set_weights(global_weights_previous_round)
        local_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        local_model.fit(client_X, client_y, epochs=num_epochs, batch_size=batch_size, verbose=0)
        
        # Tính cosine similarity giữa trọng số mô hình cục bộ và mô hình toàn cầu
        local_weights = local_model.get_weights()
        sim = cosine_similarity(local_weights, global_weights_previous_round)
        similarities.append(sim)
        local_models.append(local_model)

    # Tính giá trị trung bình của các điểm cosine similarity
    mean_similarity = np.mean(similarities)

    # Lọc và giữ lại những mô hình có similarity cao hơn giá trị trung bình
    valid_models = [local_models[i] for i, sim in enumerate(similarities) if sim >= mean_similarity]
    data_distribution = [len(client_data[i]) for i, sim in enumerate(similarities) if sim >= mean_similarity]
    # Cập nhật trọng số mô hình toàn cầu chỉ từ những mô hình hợp lệ
    print('Round ', round, ' - Evaluation:')
    print(similarities)
    print('Mean: ', mean_similarity)
    for i, sim in enumerate(similarities):
        if sim < mean_similarity:
            print('Eliminated: Node ', i)
    if valid_models:
        global_model.set_weights(fed_avg(valid_models, data_distribution))
        eval(global_model)
    else:
        print('ERR')
    # Lưu lại trọng số mới cho vòng lặp tiếp theo
    global_weights_previous_round = global_model.get_weights()


Round  0  - Evaluation:
[0.9257112, 0.9448498, 0.9533578, 0.92191905]
Mean:  0.9364595
Eliminated: Node  0
Eliminated: Node  3
2860/2860 [==============================] - 5s 2ms/step
tf.Tensor([1 1 1 1 1 1 1 1 1 1], shape=(10,), dtype=int64)
[1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.76      0.90      0.83      2153
           1       1.00      0.99      1.00     89346

    accuracy                           0.99     91499
   macro avg       0.88      0.95      0.91     91499
weighted avg       0.99      0.99      0.99     91499

Total accuracy: 0.99 

Round  1  - Evaluation:
[0.93602586, 0.9495543, 0.96481246, 0.9353792]
Mean:  0.94644296
Eliminated: Node  0
Eliminated: Node  3
2860/2860 [==============================] - 5s 2ms/step
tf.Tensor([1 1 1 1 1 1 1 1 1 1], shape=(10,), dtype=int64)
[1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.75      0.92      0.83      2153
         

In [ ]:
"""
global_model = build_model()

for round in range(num_rounds):
    local_models = []
    cnt = 1
    for client_X, client_y in zip(client_data, client_labels):
        print('Round ', round, ' - Client ', cnt)
        # Tính số batch đầy đủ có thể có
        num_full_batches = client_X.shape[0] // batch_size
        # Cắt dữ liệu để chỉ bao gồm các batch đầy đủ
        full_batch_data_size = num_full_batches * batch_size
        client_X_full_batches = client_X[:full_batch_data_size]
        client_y_full_batches = client_y[:full_batch_data_size]

        cnt += 1
        local_model = build_model()
        local_model.set_weights(global_model.get_weights())
        
        optimizer = DPKerasSGDOptimizer(
            l2_norm_clip=l2_norm_clip,
            noise_multiplier=noise_multiplier,
            num_microbatches=1,
            learning_rate=0.001
        )

        local_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        local_model.fit(client_X_full_batches, client_y_full_batches, epochs=num_epochs, batch_size=batch_size, verbose=0)
        local_models.append(local_model)

    global_model.set_weights(fed_avg(local_models, [len(cli_dt[:full_batch_data_size]) for cli_dt in client_data]))

    print('Round ', round, ' - Evaluation:')
    eval(global_model)
"""

Round  0  - Client  1
Round  0  - Client  2
Round  0  - Client  3
Round  0  - Evaluation:
141597/141597 [==============================] - 214s 2ms/step
tf.Tensor([1 0 1 1 1 1 1 1 0 1], shape=(10,), dtype=int64)
[1 1 1 1 1 1 1 1 0 1]
              precision    recall  f1-score   support

           0       0.70      0.57      0.63    106786
           1       0.99      0.99      0.99   4424292

    accuracy                           0.98   4531078
   macro avg       0.84      0.78      0.81   4531078
weighted avg       0.98      0.98      0.98   4531078

Total accuracy: 0.98 

Round  1  - Client  1
Round  1  - Client  2
Round  1  - Client  3
Round  1  - Evaluation:
141597/141597 [==============================] - 215s 2ms/step
tf.Tensor([1 0 1 1 1 1 1 1 0 1], shape=(10,), dtype=int64)
[1 1 1 1 1 1 1 1 0 1]
              precision    recall  f1-score   support

           0       0.59      0.82      0.69    106786
           1       1.00      0.99      0.99   4424292

    accuracy      